#Using Embalance Learning(Text Augmentation) on Imbalance data to balance and perform CNN model binary classifier

In [ ]:
pip install nlpaug

     |████████████████████████████████| 410 kB 4.8 MB/s 


In [ ]:

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
#setup="from __main__ import gpu"

In [ ]:
import pandas as pd
import csv
import re
import nlpaug
import nlpaug.augmenter.word as naw

In [ ]:
import os
import math
import gc
import pickle
from keras.initializers import Constant

import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix
from keras.models import Model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import numpy as np
from csv import writer
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import metrics

#for Smote 
import imblearn
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot

In [ ]:
pip install numpy nltk gensim textblob googletrans 

     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 1.3 MB 10.7 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 65 kB 3.1 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=76d33447eaa4b8c867c0f17eb28d148d99a0e7e3c2e375bda8b5ebf16f800202
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans


In [ ]:
pip install googletrans==3.1.0a0

  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=a33959cefd8e01670856497851e4e9a24e662ad6af199f0371d30e7025c7513c
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [ ]:
pip install textaugment

In [ ]:
from textaugment import Word2vec
from textaugment import Wordnet
from textaugment import Translate


In [ ]:
from textaugment import EDA


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_mr= pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/TextAugmentation/Data/Movie_Review_Sentiment_Analysis.tsv",delimiter='\t')

In [ ]:
pd.set_option('display.max_colwidth', -1)
df_mr.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2


In [ ]:
len(df_mr)

156060

In [ ]:

def clean_str(in_str):
    in_str = str(in_str)
    # replace urls with 'url'
    in_str = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", in_str)
    in_str = re.sub(r'([^\s\w]|_)+', '', in_str)
    no_digits=[]
    for i in in_str:
      if not i.isdigit():
        no_digits.append(i)
        # Now join all elements of the list with '', 
        # which puts all of the characters together. 
    in_str = ''.join(no_digits)
    return in_str.strip().lower()


df_mr['text'] = df_mr['Phrase'].apply(clean_str)


"\n\nsample1=[]\nsamples=df['full_review_text']\nfor sample in samples:\n   sample1.append(sample.lstrip('0123456789.-/ '))\ndf['text']=sample1\n"

In [ ]:
df_mr.head(5)

,PhraseId,SentenceId,Phrase,Sentiment,text
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapades demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amounts to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapades demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [ ]:
df_mr.Sentiment.value_counts()

2    79582
3    32927
1    27273
4    9206 
0    7072 
Name: Sentiment, dtype: int64

In [ ]:
df_mr_0 = df_mr[df_mr['Sentiment'] == 0].sample(frac=1)
df_mr_1 = df_mr[df_mr['Sentiment'] == 1].sample(frac=1)
df_mr_2 = df_mr[df_mr['Sentiment'] == 2].sample(frac=1)
df_mr_3 = df_mr[df_mr['Sentiment'] == 3].sample(frac=1)
df_mr_4 = df_mr[df_mr['Sentiment'] == 4].sample(frac=1)

In [ ]:
print(len(df_mr_0))
print(len(df_mr_1))
print(len(df_mr_2))
print(len(df_mr_3))
print(len(df_mr_4))

7072
27273
79582
32927
9206


In [ ]:
sample_size=7072


In [ ]:
#data = pd.concat([df_mr_0.head(len(df_mr_0)), df_mr_1.head(len(df_mr_1)), df_mr_2.head(len(df_mr_2)), df_mr_3.head(len(df_mr_3)), df_mr_4.head(len(df_mr_4))]).sample(frac=1)
data = pd.concat([df_mr_0.head(len(df_mr_0)), df_mr_1.head(len(df_mr_1)), df_mr_2.head(len(df_mr_2)), df_mr_4.head(len(df_mr_4))]).sample(frac=1)

In [ ]:
data

,PhraseId,SentenceId,Phrase,Sentiment,text,bool_raiting,raiting
136158,136159,7358,achronological,2,achronological,0,2
31827,31828,1491,undercut,1,undercut,0,1
114304,114305,6080,Jarecki and,2,jarecki and,0,2
91694,91695,4771,flawless,4,flawless,1,4
27901,27902,1281,this strenuously unconventional movie is supposed to be,2,this strenuously unconventional movie is supposed to be,0,2
...,...,...,...,...,...,...,...
58576,58577,2951,on stage,2,on stage,0,2
95950,95951,5013,a zippy jazzy score,4,a zippy jazzy score,1,4
48492,48493,2369,the most watchable film,4,the most watchable film,1,4
80613,80614,4151,The 50-something lovebirds are too immature and unappealing to care about .,0,the something lovebirds are too immature and unappealing to care about,0,0


In [ ]:
156060

156060

In [ ]:
len(df_mr)

156060

In [ ]:
df=data
df = df.reset_index()

In [ ]:
#trace only rating from the star rating and store in the column 'rating'
#create two classes 1 means positive and 0 means negative review

raiting=[]
bool_raiting=[]
for line1 in df['Sentiment']:

  #x=int(line1[1:2])
  raiting.append(line1)
  if line1>=3:
    bool_raiting.append(1)
  else:
    bool_raiting.append(0)
    
df['bool_raiting']= bool_raiting
df['raiting']=raiting
  #count+=1
pos=0
neg=0
for i in df['bool_raiting']:
  if i==1:
    pos+=1
  else:
    neg+=1
print("Total size of dataset = ",len(df['text']))
print("Positive(1) classes = ",pos)
print("Negative(0) classes = ",neg)
#sst2_data=df


Total size of dataset =  123133
Positive(1) classes =  9206
Negative(0) classes =  113927


In [ ]:
sentence_length=0
for line in df['text']:
  max=0
  for i in line:
    max+=1
  
  if(sentence_length<max):
    sentence_length=max


In [ ]:
print(sentence_length)

257


#CNN 


In [ ]:

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from keras.models import Model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import re

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
max_features = 20000 # this is the number of words we care about
sequence_length=sentence_length


In [ ]:

def embedding(text,bool_raiting,sequence_length):
  tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
  tokenizer.fit_on_texts(text.values)

  sst2_X = tokenizer.texts_to_sequences(text)
  sst2_X = pad_sequences(sst2_X, sequence_length)

  sst2_y = bool_raiting  #sst2_data['bool_raiting'].values

  embeddings_index = {}
  f = open('/content/drive/MyDrive/Colab_Notebooks/TextAugmentation/glove-global-vectors-for-word-representation/glove.6B.200d.txt')
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  word_index = tokenizer.word_index

  print('Found %s unique tokens.' % len(word_index))

  num_words = min(max_features, len(word_index)) + 1
  print(num_words)
  embedding_dim = 200 
  num_filters = 100
  # first create a matrix of zeros, this is our embedding matrix
  embedding_matrix = np.zeros((num_words, embedding_dim))

  # for each word in out tokenizer lets try to find that work in our w2v model
  for word, i in word_index.items():
      if i > max_features:
          continue
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # we found the word - add that words vector to the matrix
          embedding_matrix[i] = embedding_vector
      else:
          # doesn't exist, assign a random vector
          embedding_matrix[i] = np.random.randn(embedding_dim)
  return (sst2_X,sst2_y,embedding_matrix,num_words)
#sst2_X_train, sst2_X_test, sst2_y_train, sst2_y_test = train_test_split(sst2_X, sst2_y, test_size=0.1)

In [ ]:
def cnn_model(num_words,embedding_matrix,sequence_length):

  embedding_dim = 
  num_filters = 100

  inputs_4 = Input(shape=(sequence_length,), dtype='int32')
  embedding_layer_4 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(inputs_4)

  reshape_4 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_4)

  conv_0_4 = Conv2D(num_filters, kernel_size=(3, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)
  conv_1_4 = Conv2D(num_filters, kernel_size=(4, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)
  conv_2_4 = Conv2D(num_filters, kernel_size=(5, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)

  maxpool_0_4 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_4)
  maxpool_1_4 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_4)
  maxpool_2_4 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_4)

  concatenated_tensor_4 = Concatenate(axis=1)([maxpool_0_4, maxpool_1_4, maxpool_2_4])
  flatten_4 = Flatten()(concatenated_tensor_4)

  dropout_4 = Dropout(0.5)(flatten_4)
  # note the different activation
  output_4 = Dense(units=1, activation='sigmoid')(dropout_4)

  model = Model(inputs=inputs_4, outputs=output_4)

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  print(model.summary())
  return model

In [ ]:
def predict_result(X_test,y_test, name,size_of_data,model):
  #perform prediction
  total=len(X_test)
  
  p_hat = model.predict(X_test)
  y_hat = np.where(p_hat > 0.5, 1, 0)
  #print(y_hat)
  #tn, fp, fn, tp = confusion_matrix(y_test,y_hat).ravel()
  result=metrics.classification_report(y_test,y_hat, labels=[0,1])
  cm=metrics.confusion_matrix(y_test,y_hat, labels=[0,1])
  
  print("Confusion matrix: \n",cm)

  return result
  

In [ ]:
def my_model(emb_input, output_bool,model):
  #X= train_input_ids
  X=emb_input
  #y = pd.get_dummies(data['Sentiment']).values
  y=output_bool
  #X_np= np.array(X,dtype='int32')
  X_np= np.array(X,dtype='float32')

  
  X_train, X_test, y_train, y_test = train_test_split(X_np, y, test_size=0.1,stratify=y)
  
  print("test set size " + str(len(X_test)))
  print("Train set size " + str(len(X_train)))

  batch_size=32
  history_4 = model.fit(X_train, y_train, epochs=30, batch_size=batch_size, verbose=1)#, validation_split=0.1)
  #plot_result(history)
  return (X_test, y_test)






In [ ]:

def shuffle_data(shuf):
  shuffled = shuf.sample(frac=1)
  return shuffled

#Dealing with Minor class

In [ ]:
pos=0
neg=0

for i in df['bool_raiting']:
  if i==1:
    pos+=1
  else:
    neg+=1
print("Total size of dataset = ",len(df['text']))
print("Positive(1) classes = ",pos)
print("Negative(0) classes = ",neg)
if(neg<pos):
  minor_class=0
else:
  minor_class=1


Total size of dataset =  123133
Positive(1) classes =  9206
Negative(0) classes =  113927


In [ ]:
minor_class

1

In [ ]:
columns=['text','bool_raiting']

df_1 = pd.DataFrame(columns=columns)
df_1 = df_1.fillna(0)

In [ ]:
print(df['bool_raiting'])

0         0
1         0
2         0
3         1
4         0
         ..
123128    0
123129    1
123130    1
123131    0
123132    0
Name: bool_raiting, Length: 123133, dtype: int64


In [ ]:
#separate the dataset of minor class to perform data balancing
count=0
#i=0
text_1=[]
bool_raiting1=[]  #
bool_raiting=df['bool_raiting']
text=df['text']
for i in range(len(df['text'])):
    
  if int(bool_raiting[i])==minor_class:
    #print("text ",i," =",text[i])
    #print(df['bool_raiting'][i])
    #df_1['bool_raiting'][i]=0
    bool_raiting1.append(minor_class)
    text_1.append(text[i])
    count+=1
df_1['text']=text_1
df_1['bool_raiting']=bool_raiting1
#error to show there is a problem
print(df_1)


                                                                                                                                                                      text  \
0     flawless                                                                                                                                                               
1     only with muscles and a lot more smarts  but just as endearing and easy to watch                                                                                       
2     a tribute to shyamalan s gifts  which are such that we ll keep watching the skies for his next project                                                                 
3     it s a very valuable film                                                                                                                                              
4     s a movie that accomplishes so much that one viewing ca nt possibly be enough                                               

In [ ]:
if pos>neg:
  difference_between_classes= pos-neg
  div=neg
else:
  difference_between_classes=neg-pos
  div=pos
print(difference_between_classes)

104721


In [ ]:
num_word_sy=2
num_synth_sentence=difference_between_classes/div
#num_synth_sentence=3.55
print(num_synth_sentence)
round_num_synth_sentence=int(round(num_synth_sentence, 0))
print(round_num_synth_sentence)
print("total number of synthetic sentence will be created ",round_num_synth_sentence*div)

11.375298718227244
11
total number of synthetic sentence will be created  101266


#Original Data 

In [ ]:
df1=df
shuffled= shuffle_data(df1)

In [ ]:
#Original Data
X1,y1,embedding_matrix1,num_words1 =embedding(shuffled['text'],shuffled['bool_raiting'].values,sequence_length)#original data

Found 400000 word vectors.
Found 16185 unique tokens.
16186


In [ ]:
columns=['embedding','bool_raiting']
original_df = pd.DataFrame(columns=columns)
original_df = original_df.fillna(0)

In [ ]:
model1= cnn_model(num_words1,embedding_matrix1,sequence_length)

X_test1, y_test1=my_model(X1, y1,model1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 257)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 257, 200)     3237200     ['input_2[0][0]']                
                                                                                                  
 reshape_1 (Reshape)            (None, 257, 200, 1)  0           ['embedding_1[0][0]']            
                                                                                                  
 conv2d_3 (Conv2D)              (None, 255, 1, 100)  60100       ['reshape_1[0][0]']              
                                                                                            

KeyboardInterrupt: ignored

In [ ]:
size_of_data1= len(df['text'])
original_result =predict_result(X_test1,y_test1, 'Original',size_of_data1,model1)
print(original_result)
#accuracy_score(sst2_y_test, list(map(lambda v: v > 0.5, y_hat_4)))

#Synthetic data using Text Augmentation

#EmbeddingAugmenter 

WordNet-based augmentation

In [ ]:
v = True # enable verbs augmentation. By default is True.
n = False # enable nouns augmentation. By default is False.
runs = 1 # number of times to augment a sentence. By default is 1.
p = 0.5 # The probability of success of an individual trial. (0.1<p<1.0), default is 0.5. Used by Geometric distribution to selects words from a sentence.
t = Wordnet(v=True ,n=True, p=0.5,runs=2)
wordnet_text=[]
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    wordnet_text.append(t.augment(line))
print(len(wordnet_text))
#print(wordnet_text)
#In the afternoon, Joseph is going to town.

100%|██████████| 9206/9206 [00:08<00:00, 1138.25it/s]

138090


In [ ]:
wordnet_text

['this quiet introspective and entertaining independent embody worth essay',
 'quiet introspective and entertaining independent',
 'nurse',
 'equal worth search',
 'a positively thrilling combination of ethnography and all the intrigue treachery fraudulence and slaying of a shakespearean tragedy or a juicy max opera',
 'a positively thrilling combining of ethnography and all the intrigue perfidy dissembling and murder',
 'thrilling',
 'a comedydrama of nearly epic dimension rout in a sincere functioning by the title character undergoing midlife crisis',
 'nearly epos',
 'root in a sincere execution by the title fibre undergoing midlife crisis',
 'in a sincere functioning',
 'a sincere performance',
 'sincere execution',
 'recommend snow dog',
 'high mirth',
 'the performances exist an absolute delight',
 'embody an absolute pleasure',
 'an absolute joyousness',
 'delight',
 'extravagant',
 'better',
 'this one equal a sweet and modest and ultimately succeed story',
 'live a sweet and m

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
wordnet_text_combine= df['text'].tolist()
wordnet_bool_raiting=[]
for i in df['bool_raiting']:
  wordnet_bool_raiting.append(i)
print(wordnet_bool_raiting)

for i in wordnet_text:
  wordnet_text_combine.append(i)
  wordnet_bool_raiting.append(minor_class)
  #count+=1


[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
columns=['text','bool_raiting']
df_wordnet_synth = pd.DataFrame(columns=columns)
df_wordnet_synth = df_wordnet_synth.fillna(0)
df_wordnet_synth['text']=wordnet_text_combine
df_wordnet_synth['bool_raiting']=wordnet_bool_raiting

In [ ]:
df_wordnet_synth.head(5)

,text,bool_raiting
0,a fan film that for the uninitiated plays better on video with the sound,0
1,in the long dishonorable history of quickie teenpop exploitation like mike stands out for its only partly synthetic decency,0
2,seen,0
3,tissues,0
4,of one recent chinese immigrant s experiences in new york city,0


In [ ]:
df_wordnet_synth=shuffle_data(df_wordnet_synth)

In [ ]:
#wordnet Data
X_wn,y_wn,embedding_matrix_wn,num_words_wn =embedding(df_wordnet_synth['text'],df_wordnet_synth['bool_raiting'].values,sequence_length)#wordnet data

Found 400000 word vectors.
Found 20029 unique tokens.
20001


In [ ]:
model_wn=cnn_model(num_words_wn,embedding_matrix_wn,sequence_length)
X_test_wn, y_test_wn=my_model(X_wn, y_wn,model_wn)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 52)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 52, 200)      4000200     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 52, 200, 1)   0           ['embedding[0][0]']              
                                                                                                  
 conv2d (Conv2D)                (None, 50, 1, 100)   60100       ['reshape[0][0]']                
                                                                                              

In [ ]:
size_of_data_wn= len(X_wn)
Wordnet_result= predict_result(X_test_wn,y_test_wn, 'wordnet',size_of_data_wn,model_wn)

print(Wordnet_result)


Confusion matrix: 
 [[13707   978]
 [ 1737 12993]]
              precision    recall  f1-score   support

           0       0.89      0.93      0.91     14685
           1       0.93      0.88      0.91     14730

    accuracy                           0.91     29415
   macro avg       0.91      0.91      0.91     29415
weighted avg       0.91      0.91      0.91     29415



RTT-based augmentation

In [ ]:
src = "en" # source language of the sentence
to = "fr" # target language
from textaugment import Translate
rtt = Translate(src="en", to="fr")
rtt_text=[]
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    rtt_text.append(rtt.augment(line))
print(len(rtt_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

 77%|███████▋  | 7047/9206 [05:10<01:35, 22.68it/s]


HTTPError: ignored

In [ ]:
rtt_text

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
rtt_text_combine= df['text'].tolist()
rtt_bool_raiting=[]
for i in df['bool_raiting']:
  rtt_bool_raiting.append(i)
print(rtt_bool_raiting)

for i in rtt_text:
  rtt_text_combine.append(i)
  rtt_bool_raiting.append(minor_class)
  #count+=1


In [ ]:
columns=['text','bool_raiting']
df_rtt_synth = pd.DataFrame(columns=columns)
df_rtt_synth = df_rtt_synth.fillna(0)
df_rtt_synth['text']=rtt_text_combine
df_rtt_synth['bool_raiting']=rtt_bool_raiting

In [ ]:
df_rtt_synth.head(5)

In [ ]:
df_rtt_synth=shuffle_data(df_rtt_synth)

In [ ]:
#RTT Data
X_rtt,y_rtt,embedding_matrix_rtt,num_words_rtt =embedding(df_rtt_synth['text'],df_rtt_synth['bool_raiting'].values,sequence_length)#RTT data

In [ ]:
model_rtt=cnn_model(num_words_rtt,embedding_matrix_rtt,sequence_length)
X_test_rtt, y_test_rtt=my_model(X_rtt, y_rtt,model_rtt)

In [ ]:
size_of_data_rtt= len(X_rtt)
rtt_result =predict_result(X_test_rtt,y_test_rtt, 'RTT Based',size_of_data_rtt,model_rtt)
print(rtt_result)


#EDA: Easy data augmentation techniques for boosting performance on text classification tasks

Synonym Replacement

In [ ]:
sr_text=[]
sr = EDA()
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    sr_text.append(sr.synonym_replacement(line,2))
print(len(sr_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

100%|██████████| 9206/9206 [00:01<00:00, 5750.14it/s]

138090


In [ ]:
sr_text

['this quiet introspective and entertaining free lance is worth attempt',
 'hushed introspective and entertaining self employed person',
 'think of',
 'is deserving essay',
 'a positively thrilling compounding of ethnography and all the intrigue perfidy deceit and murder of a shakespearean tragedy or a juicy soap opera',
 'a positively thrill combination of ethnography and all the intrigue perfidy deceit and murder',
 'beatify',
 'a comedydrama of nearly epic symmetry rooted in a sincere performance by the deed character undergoing midlife crisis',
 'almost epic poem',
 'rout in a sincere performance by the title character undergo midlife crisis',
 'in a earnest carrying into action',
 'a solemn public presentation',
 'earnest operation',
 'commend c p snow dogs',
 'gamey glee',
 'the public presentation are an absolute joyousness',
 'are an inviolable rejoice',
 'an right down pleasure',
 'pleasure',
 'spendthrift',
 'ameliorate',
 'this one is a sweet and modest and ultimately acquir

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
sr_text_combine= df['text'].tolist()
sr_bool_raiting=[]
for i in df['bool_raiting']:
  sr_bool_raiting.append(i)
print(sr_bool_raiting)

for i in sr_text:
  sr_text_combine.append(i)
  sr_bool_raiting.append(minor_class)
  #count+=1


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
columns=['text','bool_raiting']
df_sr_synth = pd.DataFrame(columns=columns)
df_sr_synth = df_sr_synth.fillna(0)
df_sr_synth['text']=sr_text_combine
df_sr_synth['bool_raiting']=sr_bool_raiting

In [ ]:
df_sr_synth.head(5)

,text,bool_raiting
0,enters a realm where few nonporn films venture and comes across as darkly funny energetic and surprisingly gentle,1
1,judicious editing,0
2,her bjorkness,0
3,old gags,0
4,trying to go,0


In [ ]:
df_sr_synth=shuffle_data(df_sr_synth)

In [ ]:
#Synonym replacement Data
X_sr,y_sr,embedding_matrix_sr,num_words_sr =embedding(df_sr_synth['text'],df_sr_synth['bool_raiting'].values,sequence_length)#Synonym replacement data

Found 400000 word vectors.
Found 23707 unique tokens.
20001


In [ ]:
model_sr=cnn_model(num_words_sr,embedding_matrix_sr,sequence_length)
X_test_sr, y_test_sr=my_model(X_sr, y_sr,model_sr)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 52)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 52, 200)      4000200     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 52, 200, 1)   0           ['embedding[0][0]']              
                                                                                                  
 conv2d (Conv2D)                (None, 50, 1, 100)   60100       ['reshape[0][0]']                
                                                                                              

In [ ]:
size_of_data_sr= len(X_sr)
sr_result =predict_result(X_test_sr,y_test_sr, 'Synonyms replacement',size_of_data_sr,model_sr)
print(sr_result)


Confusion matrix: 
 [[13731   954]
 [ 1442 13288]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     14685
           1       0.93      0.90      0.92     14730

    accuracy                           0.92     29415
   macro avg       0.92      0.92      0.92     29415
weighted avg       0.92      0.92      0.92     29415



Random Deletion

In [ ]:
rd = EDA()
rd_text=[]
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    rd_text.append(rd.random_deletion(line, p=0.2))
print(len(rd_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

100%|██████████| 9206/9206 [00:00<00:00, 129366.82it/s]

138090


In [ ]:
rd_text

['quiet introspective and entertaining independent is worth',
 'introspective and entertaining',
 ['entertaining'],
 'is worth seeking',
 'a positively of ethnography and all the betrayal deceit and murder of a shakespearean tragedy a juicy soap',
 'a positively combination of ethnography and all the intrigue betrayal deceit and murder',
 ['thrilling'],
 'a comedydrama nearly epic proportions in a sincere performance by the title character undergoing midlife',
 'epic',
 'rooted in a performance by the title character undergoing midlife crisis',
 'in a sincere performance',
 'a performance',
 'sincere performance',
 'recommend snow dogs',
 'high hilarity',
 'performances are absolute joy',
 'are an absolute joy',
 'an absolute',
 ['joy'],
 ['extravagant'],
 ['better'],
 'this is a sweet and modest and ultimately winning',
 'sweet and and ultimately',
 'a and modest',
 ['sweet'],
 'ultimately winning story',
 'gorgeous',
 'love',
 ['love'],
 'best indie of the year',
 ['best'],
 'of year

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
rd_text_combine= df['text'].tolist()
rd_bool_raiting=[]
for i in df['bool_raiting']:
  rd_bool_raiting.append(i)
print(rd_bool_raiting)

for i in rd_text:
  rd_text_combine.append(i)
  rd_bool_raiting.append(minor_class)
  #count+=1


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
columns=['text','bool_raiting']
df_rd_synth = pd.DataFrame(columns=columns)
df_rd_synth = df_rd_synth.fillna(0)
df_rd_synth['text']=rd_text_combine
df_rd_synth['bool_raiting']=rd_bool_raiting

In [ ]:
df_rd_synth.head(5)

,text,bool_raiting
0,enters a realm where few nonporn films venture and comes across as darkly funny energetic and surprisingly gentle,1
1,judicious editing,0
2,her bjorkness,0
3,old gags,0
4,trying to go,0


In [ ]:
df_rd_synth=shuffle_data(df_rd_synth)

In [ ]:
#Random Delete Data
X_rd,y_rd,embedding_matrix_rd,num_words_rd =embedding(df_rd_synth['text'],df_rd_synth['bool_raiting'].values,sequence_length)#Random Delete data

Found 400000 word vectors.
Found 16213 unique tokens.
16214


In [ ]:
model_rd=cnn_model(num_words_rd,embedding_matrix_rd,sequence_length)
X_test_rd, y_test_rd=my_model(X_rd, y_rd,model_rd)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 52)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 52, 200)      3242800     ['input_2[0][0]']                
                                                                                                  
 reshape_1 (Reshape)            (None, 52, 200, 1)   0           ['embedding_1[0][0]']            
                                                                                                  
 conv2d_3 (Conv2D)              (None, 50, 1, 100)   60100       ['reshape_1[0][0]']              
                                                                                            

In [ ]:
size_of_data_rd= len(X_rd)
rd_result =predict_result(X_test_rd,y_test_rd, 'Random Deletion',size_of_data_rd,model_rd)
print(rd_result)


Confusion matrix: 
 [[12839  1846]
 [ 2505 12225]]
              precision    recall  f1-score   support

           0       0.84      0.87      0.86     14685
           1       0.87      0.83      0.85     14730

    accuracy                           0.85     29415
   macro avg       0.85      0.85      0.85     29415
weighted avg       0.85      0.85      0.85     29415



Random Swap

In [ ]:
rs_text=[]
rs = EDA()
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    rs_text.append(rs.random_swap(line))
print(len(rs_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

100%|██████████| 9206/9206 [00:00<00:00, 126915.05it/s]

138090


In [ ]:
rs_text

['this introspective quiet and entertaining independent is worth seeking',
 'and introspective quiet entertaining independent',
 'entertaining',
 'worth is seeking',
 'and positively thrilling combination of ethnography and all the intrigue betrayal deceit a murder of a shakespearean tragedy or a juicy soap opera',
 'a positively thrilling combination of ethnography intrigue all the and betrayal deceit and murder',
 'thrilling',
 'performance comedydrama of nearly epic proportions rooted in a sincere a by the title character undergoing midlife crisis',
 'epic nearly',
 'rooted in a midlife performance by the title character undergoing sincere crisis',
 'performance a sincere in',
 'performance sincere a',
 'performance sincere',
 'snow recommend dogs',
 'hilarity high',
 'are performances the an absolute joy',
 'joy an absolute are',
 'joy absolute an',
 'joy',
 'extravagant',
 'better',
 'this one is a sweet ultimately modest and and winning story',
 'is and sweet and modest a ultimat

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
rs_text_combine= df['text'].tolist()
rs_bool_raiting=[]
for i in df['bool_raiting']:
  rs_bool_raiting.append(i)
print(rs_bool_raiting)

for i in rs_text:
  rs_text_combine.append(i)
  rs_bool_raiting.append(minor_class)
  #count+=1


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
columns=['text','bool_raiting']
df_rs_synth = pd.DataFrame(columns=columns)
df_rs_synth = df_rs_synth.fillna(0)
df_rs_synth['text']=rs_text_combine
df_rs_synth['bool_raiting']=rs_bool_raiting

In [ ]:
df_rs_synth.head(5)

,text,bool_raiting
0,enters a realm where few nonporn films venture and comes across as darkly funny energetic and surprisingly gentle,1
1,judicious editing,0
2,her bjorkness,0
3,old gags,0
4,trying to go,0


In [ ]:
df_rs_synth=shuffle_data(df_rs_synth)

In [ ]:
#Random swap Data
X_rs,y_rs,embedding_matrix_rs,num_words_rs =embedding(df_rs_synth['text'],df_rs_synth['bool_raiting'].values,sequence_length)#random swap data

Found 400000 word vectors.
Found 16213 unique tokens.
16214


In [ ]:
model_rs=cnn_model(num_words_rs,embedding_matrix_rs,sequence_length)
X_test_rs, y_test_rs=my_model(X_rs, y_rs,model_rs)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 52)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 52, 200)      3242800     ['input_3[0][0]']                
                                                                                                  
 reshape_2 (Reshape)            (None, 52, 200, 1)   0           ['embedding_2[0][0]']            
                                                                                                  
 conv2d_6 (Conv2D)              (None, 50, 1, 100)   60100       ['reshape_2[0][0]']              
                                                                                            

In [ ]:
size_of_data_rs= len(X_rs)
rs_result =predict_result(X_test_rs,y_test_rs, 'Random Swap',size_of_data_rs,model_rs)
print(rs_result)


Confusion matrix: 
 [[12715  1970]
 [  564 14166]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91     14685
           1       0.88      0.96      0.92     14730

    accuracy                           0.91     29415
   macro avg       0.92      0.91      0.91     29415
weighted avg       0.92      0.91      0.91     29415



Random Insertion

In [ ]:
"""
ri_text=[]
ri = EDA()
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    ri_text.append(ri.random_insertion(line))
print(len(ri_text))
"""
ri_text=[]
ri = EDA()
for i in range(round_num_synth_sentence):
  for line in tqdm(df_1['text']):
    #print(line)
    if line=='no':
      continue
    else:
      ri_text.append(ri.random_insertion(line))
print(len(ri_text))

#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town
#out=t.augment('In the afternoon, John is going to town')
#print(out)
#In the afternoon John goes to town

100%|██████████| 9206/9206 [00:00<00:00, 9318.72it/s]

138075


In [ ]:
ri_text

['this self examining quiet introspective and entertaining independent is worth seeking',
 'toy with quiet introspective and entertaining independent',
 'toy with entertaining',
 'is worth try seeking',
 'a positively thrilling combination of ethnography deception and all the intrigue betrayal deceit and murder of a shakespearean tragedy or a juicy soap opera',
 'a positively thrilling combination of ethnography and exhilarate all the intrigue betrayal deceit and murder',
 'exhilarate thrilling',
 'carrying out a comedydrama of nearly epic proportions rooted in a sincere performance by the title character undergoing midlife crisis',
 'nearly epical epic',
 'rooted in a sincere performance by the title character deed of conveyance undergoing midlife crisis',
 'in a solemn sincere performance',
 'solemn a sincere performance',
 'solemn sincere performance',
 'recommend snow blow dogs',
 'high high school hilarity',
 'delight the performances are an absolute joy',
 'are an absolute out an

In [ ]:
#append both original and synthetic sentence in a single data frame (df)
ri_text_combine= df['text'].tolist()
ri_bool_raiting=[]
for i in df['bool_raiting']:
  ri_bool_raiting.append(i)
print(ri_bool_raiting)

for i in ri_text:
  ri_text_combine.append(i)
  ri_bool_raiting.append(minor_class)
  #count+=1


[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
columns=['text','bool_raiting']
df_ri_synth = pd.DataFrame(columns=columns)
df_ri_synth = df_ri_synth.fillna(0)
df_ri_synth['text']=ri_text_combine
df_ri_synth['bool_raiting']=ri_bool_raiting

In [ ]:
df_ri_synth.head(5)

,text,bool_raiting
0,one more member,0
1,with the warning,0
2,died a matter of weeks,0
3,analyze,0
4,heroes would be a film that is nt this painfully forced false and fabricated,0


In [ ]:
df_ri_synth=shuffle_data(df_ri_synth)

In [ ]:
#Random insert Data
X_ri,y_ri,embedding_matrix_ri,num_words_ri =embedding(df_ri_synth['text'],df_ri_synth['bool_raiting'].values,sequence_length)#random insert data

Found 400000 word vectors.
Found 17974 unique tokens.
17975


In [ ]:
model_ri=cnn_model(num_words_ri,embedding_matrix_ri,sequence_length)
X_test_ri, y_test_ri=my_model(X_ri, y_ri,model_ri)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 52)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 52, 200)      3595000     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 52, 200, 1)   0           ['embedding[0][0]']              
                                                                                                  
 conv2d (Conv2D)                (None, 50, 1, 100)   60100       ['reshape[0][0]']                
                                                                                              

In [ ]:
size_of_data_ri= len(X_ri)
ri_result =predict_result(X_test_ri,y_test_ri, 'Random Insertion',size_of_data_ri,model_ri)
print(ri_result)


Confusion matrix: 
 [[13553  1133]
 [  617 14111]]
              precision    recall  f1-score   support

           0       0.96      0.92      0.94     14686
           1       0.93      0.96      0.94     14728

    accuracy                           0.94     29414
   macro avg       0.94      0.94      0.94     29414
weighted avg       0.94      0.94      0.94     29414



#End